In [3]:
import tensorflow as tf
import keras.backend as K
import numpy as np
import pandas as pd

from keras.utils import multi_gpu_model
from keras.preprocessing.image import ImageDataGenerator
from utils import load_pretrained_weights

In [4]:
K.clear_session()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

In [15]:
train_dir = "/mnt/safe01/data/train/"
validation_dir = "/mnt/safe01/data/validation/"
test_dir = "/mnt/safe01/data/test/"
weights_path = "/home/nhannguyen/sigprocsoccomp/model_weights/inception_resnet_v2/"

In [55]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    batch_size=36,
                                                    target_size=(512, 512),
                                                    shuffle=False)

Found 22319 images belonging to 10 classes.


In [8]:
inceptionresnetv2 = load_pretrained_weights('InceptionResNetV2', input_shape=(512, 512, 3))
inceptionresnetv2.load_weights(weights_path + 'multigpu.hdf5')

In [9]:
parallel_model = multi_gpu_model(model=inceptionresnetv2, gpus=4)

In [10]:
def predict_image_generator(path, model, batch_size, num_test_samples, class_indices):
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    test_generator = test_datagen.flow_from_directory(path,
                                                      batch_size=batch_size,
                                                      target_size=(512, 512),
                                                      shuffle=False)
    y_pred = model.predict_generator(test_generator, steps=num_test_samples / batch_size,
                                     verbose=1)
    indices_pred = np.argmax(y_pred, axis=1)
    indices_class = {v: k for k, v in class_indices.items()}
    class_pred = np.vectorize(indices_class.get)(indices_pred)
    return class_pred

class_pred = predict_image_generator(path=test_dir, model=parallel_model,
                                     batch_size=40, num_test_samples=2640,
                                     class_indices=train_generator.class_indices)

Found 2640 images belonging to 1 classes.
66/66 [==============================] - 36s 544ms/step


In [56]:
def make_submission(input_path, output_path, camera_prediction):
    """
    Concatenate the test images names with their associated camera prediction in a single
    DataFrame.
    """
    import os
    all_test_img_names = sorted(os.listdir(input_path))
    all_test_img_renamed = pd.Series([fname.replace('.jpg', '.tif') for 
                                      fname in all_test_img_names])
    camera = pd.Series(camera_prediction)
    df = pd.concat([all_test_img_renamed, camera], axis=1)
    df.columns = ["fname", "camera"]
    df.to_csv(output_path, index=False)
    return df

make_submission(input_path=test_dir+'/test',
                output_path="/home/nhannguyen/sigprocsoccomp/submission.csv",
                camera_prediction=class_pred).head()

,fname,camera
0,img_0002a04_manip.tif,Motorola-X
1,img_001e31c_unalt.tif,Motorola-Droid-Maxx
2,img_00275cf_manip.tif,iPhone-6
3,img_0034113_unalt.tif,Samsung-Galaxy-S4
4,img_00344b7_unalt.tif,Samsung-Galaxy-Note3


In [18]:
validation_datagen = ImageDataGenerator(rescale=1. / 255)    
validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              batch_size=36,
                                                              target_size=(512, 512),
                                                              shuffle=False)

Found 2430 images belonging to 10 classes.


In [51]:
inceptionresnetv2 = load_pretrained_weights('InceptionResNetV2', input_shape=(512, 512, 3))
inceptionresnetv2.load_weights(weights_path + 'multigpu.hdf5')

In [52]:
def create_features_extractor(model, num_layers_removed, ix_first_removed_layer):
    for i in range(num_layers_removed):
        model.layers.pop(ix_first_removed_layer)
    return model


features_extractor = create_features_extractor(model=inceptionresnetv2,
                                               num_layers_removed=7,
                                               ix_first_removed_layer=783)

In [54]:
parallel_features_extractor = multi_gpu_model(features_extractor, gpus=4)

In [58]:
X_train = parallel_features_extractor.predict_generator(train_generator,
                                                        steps=22319//36+1,
                                                        verbose=1)

620/620 [==============================] - 748s 1s/step


In [60]:
X_train.shape

(22319, 10)